# Anatomy of an MD simulation script File

Different simulation packages often use different languages and have different syntax. For example, HOOMD-Blue uses a Python based interface, LAMMPS uses its own custom scripting language, and GROMACS relies upon an input file to define parameters and methods.   However, despite these differences they all generally require the same information to be passed to the code 

#### Basic components of most script/input files
- system/code initialization 
>box size, particle types, particle initial positions
- interaction definition 
>how do the different species interact with each other
- integrator setup
>what algorithm will we use to advance particles in time, time step of integration, thermodynamic state point (i.e., T or P).
- runtime parameters 
>total simulation time, which quantities to output and how frequently

## Basic HOOMD-Blue script file
Note, this is currently not setup to be runable. This is currently based on this tutorial:
http://nbviewer.jupyter.org/github/joaander/hoomd-examples/blob/master/Tutorial%20-%20MD%20-%20Lennard%20Jones.ipynb

### Initialization

HOOMD-Blue uses a Python interface, so we must first import the library. 

`import hoomd`<br>
`import hoomd.md`<br>

The next bit is used to tell where HOOMD will execute the simulation (GPU or CPU). GPU is default if it exists, unless CPU specified via a commandline call.

`hoomd.context.initialize("")`

Particle positions next need to be specified.  HOOMD includes a few helper functions, primarily for the purposes of benchmarking.  E.g., creating a $n$ by $n$ by $n$ lattice of particles.

`hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)`

Note, by defaul, these will be labeled as type "A".

### Interaction Definition

We must define how particles interact.  In HOOMD, when defining pair potential, a neighborlist needs to be passed to it. 

`nl = hoomd.md.nlist.cell()`<br>
`lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)`

The individual epsilon and sigma parameters can then be set for each pair of interactions (here we only have type A):

`lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)`



### Integrator Setup

To actually move particles through time, we'll need to specify the timestep (i.e., our time resolution for which we will numerically integrate):

`hoomd.md.integrate.mode_standard(dt=0.005)`

Next we specify the algorithm and which particles it will apply to.  For example, here we create a group named "all" and use the langevin integration method:

`all = hoomd.group.all()`<br>
`hoomd.md.integrate.langevin(group=all, kT=0.2, seed=42)`

### Runtime parameters

First, let us define a log which will record various thermodynamic quantities at set intervals:

`hoomd.analyze.log(filename="log-output.log",`<br>
`                  quantities=['potential_energy', 'temperature'],`<br>
`                  period=100,`<br>
`                  overwrite=True);`<br>

We can also specify trajectory outputs

`hoomd.dump.gsd("trajectory.gsd", period=2e3, group=all, overwrite=True)`

Finally, specify how long we want to run:

`hoomd.run(1e4);

### Full script

In [1]:
import hoomd
import hoomd.md

hoomd.context.initialize("");
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);

nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);

hoomd.md.integrate.mode_standard(dt=0.005);

all = hoomd.group.all();
hoomd.md.integrate.langevin(group=all, kT=0.2, seed=42);

hoomd.analyze.log(filename="log-output.log",
                  quantities=['potential_energy', 'temperature'],
                  period=100,
                  overwrite=True);
hoomd.dump.gsd("trajectory.gsd", period=2e3, group=all, overwrite=True);

hoomd.run(1e4);

ModuleNotFoundError: No module named 'hoomd'